In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import outlines
from PIL import Image

from transformers import AutoProcessor, AutoModelForImageTextToText

/home/fbruzzesi/open-source/moin-moin/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForImageTextToText

tokenizer = AutoTokenizer.from_pretrained("facebook/nougat-small")
model = AutoModelForImageTextToText.from_pretrained("facebook/nougat-small")

Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    896,
    672
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "qkv_bias": true,
  "transformers_version": "4.46.3",
  "use_absolute_embeddings": false,
  "window_size": 7
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "add_final_layer_norm": true

In [11]:
model_name="Qwen/Qwen2-VL-2B-Instruct"
model_class=AutoModelForImageTextToText

def get_vision_model(model_name: str, model_class):
    model_kwargs = {
        "torch_dtype": torch.bfloat16,
        "device_map": "auto",
    }
    processor_kwargs = {
        "device": "cpu"
    }

    model = outlines.models.transformers_vision(
        model_name = model_name,
        model_class = model_class,
        model_kwargs = model_kwargs,
        processor_kwargs = processor_kwargs,
    )
    return model
model = get_vision_model(model_name, model_class)

Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    896,
    672
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "qkv_bias": true,
  "transformers_version": "4.46.3",
  "use_absolute_embeddings": false,
  "window_size": 7
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "add_final_layer_norm": true

ValueError: VisionEncoderDecoderModel does not support `device_map='auto'`. To implement support, the model class needs to implement the `_no_split_modules` attribute.

In [5]:
@outlines.prompt
def customer_support(request):
    """You are an experienced government officer.

    Given an image and possibly additional text information, you need to determine
    to which category the issue belongs to.

    The descriptions of the issues are:

    - Infrastructure: Infrastructure problems significantly impact the safety and accessibility of public spaces. This category includes conditions related to roads, bridges, sidewalks, and public transportation systems.
    - Safety Concerns: Public safety is a critical aspect of community well-being, and issues in this category often lead to residents feeling insecure in their environment.
    - Waste Management and Cleanliness: Maintaining cleanliness in public spaces is essential for community health and aesthetics.
    - Public Amenities and Facilities: The availability and condition of public amenities significantly impact the quality of life in a community.

    # TASK

    Request: {{ request }}
    Label: """

The descriptions of the issues are:

    - Infrastructure: Infrastructure problems significantly impact the safety and accessibility of public spaces.
        This category includes conditions related to roads, bridges, sidewalks, and public transportation systems. Common issues reported in this area include:
            * Potholes: Large, deep holes in the road surface can cause damage to vehicles and pose a hazard to cyclists and pedestrians.
            * Broken Traffic Signals: Non-functional traffic lights can lead to confusion and accidents at busy intersections, endangering both drivers and pedestrians.
            * Damaged Sidewalks: Cracked or uneven sidewalks can make it difficult for individuals with mobility challenges to navigate, increasing the risk of trips and falls.
            * Unsafe Pedestrian Crossings: Lack of proper signage, faded crosswalks, or missing curb ramps can create dangerous situations for pedestrians attempting to cross busy streets.
    - Safety Concerns: Public safety is a critical aspect of community well-being, and issues in this category often lead to residents feeling insecure in their environment.
        Typical problems include:
            * Inadequate Street Lighting: Poorly lit streets and parks can increase the risk of crime and accidents, making residents feel unsafe when walking at night.
            * Vandalism: Graffiti, broken windows, and damaged property can create a sense of neglect and fear in neighborhoods, prompting residents to report these issues.
            * Insufficient Police Presence: Areas with high crime rates may require increased law enforcement visibility to deter criminal activity and reassure the community.
            * Emergency Services Accessibility: Delays in emergency response due to blocked access or poorly marked emergency routes can endanger lives during critical situations.

    - Waste Management and Cleanliness: Maintaining cleanliness in public spaces is essential for community health and aesthetics.
        Problems in this category often lead to public complaints and reports, such as:
            * Overflowing Trash Bins: When public trash cans are not emptied regularly, they can overflow, leading to littering and attracting pests.
            * Illegal Dumping: Unlawful disposal of waste in public areas, such as vacant lots or parks, can create environmental hazards and degrade community appearance.
            * Graffiti: While some may consider it art, unwanted graffiti can contribute to a sense of disorder and neglect in public spaces, prompting calls for removal.
            * Lack of Regular Cleaning: Parks, streets, and public plazas that are not maintained can accumulate debris, making them uninviting and potentially hazardous for visitors.

    - Public Amenities and Facilities: The availability and condition of public amenities significantly impact the quality of life in a community.
        Issues in this category often involve:
            * Broken Playground Equipment: Damaged swings, slides, or other play structures can pose safety risks for children and limit recreational opportunities.
            * Inaccessible Public Restrooms: Facilities that are poorly maintained, lack supplies, or are not accessible to individuals with disabilities can deter people from using public spaces.
            * Insufficient Seating: In parks or public squares, a lack of benches or seating areas can make it difficult for people to relax and enjoy their surroundings.
            * Neglected Recreational Areas: Sports fields and community centers that are not properly maintained can discourage community engagement and physical activity.

In [6]:
from outlines.samplers import greedy

generator = outlines.generate.choice(
    model,
    choices=[
        "Infrastructure",
        "Safety Concerns",
        "Waste Management and Cleanliness",
        "Public Amenities and Facilities",
    ],
    sampler=greedy()
)

In [7]:
requests = [
    "My hair is one fire! Please help me!!!",
]

images = [
    [Image.open('data/test.jpeg').resize((256, 256))]
]


In [8]:
instructions = """
You are an experienced government officer.

Given an image and possibly additional text information, you need to determine
to which category the issue belongs to.

The descriptions of the issues are:

- Infrastructure: Infrastructure problems significantly impact the safety and accessibility of public spaces. This category includes conditions related to roads, bridges, sidewalks, and public transportation systems.
- Safety Concerns: Public safety is a critical aspect of community well-being, and issues in this category often lead to residents feeling insecure in their environment.
- Waste Management and Cleanliness: Maintaining cleanliness in public spaces is essential for community health and aesthetics.
- Public Amenities and Facilities: The availability and condition of public amenities significantly impact the quality of life in a community.
"""

In [9]:
# prompts = [customer_support(request) for request in requests]
labels = generator([instructions], images)
print(labels)


/home/fbruzzesi/open-source/moin-moin/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/fbruzzesi/open-source/moin-moin/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


ValueError: Image features and image tokens do not match: tokens: 0, features 81